# Connecting to Eikon Data API

In [53]:
import refinitiv.data as rd
import refinitiv.data.eikon as ek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_opt

from pprint import pprint
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [54]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x117903910 {name='workspace'}>

In [55]:
# Get constituents of the FTSE 100 index
constituents_data = ek.get_data(
    instruments=['0#.FTSE'],
    fields=['TR.CommonName', 'TR.GICSSectorName', 'TR.PrimaryQuote']
)[0]  # [0] to get the DataFrame from the result tuple

print(constituents_data)

   Instrument              Company Common Name Primary Quote RIC
0      STAN.L           Standard Chartered PLC            STAN.L
1      CRDA.L          Croda International PLC            CRDA.L
2      ANTO.L                  Antofagasta PLC            ANTO.L
3       EZJ.L                      Easyjet PLC             EZJ.L
4      BNZL.L                        Bunzl plc            BNZL.L
..        ...                              ...               ...
95     ULVR.L                     Unilever PLC            ULVR.L
96     OCDO.L                  Ocado Group PLC            OCDO.L
97     LSEG.L  London Stock Exchange Group PLC            LSEG.L
98     TSCO.L                        Tesco PLC            TSCO.L
99     LGEN.L        Legal & General Group PLC            LGEN.L

[100 rows x 3 columns]


In [56]:
ind_const, err = ek.get_data(
        instruments = constituents_data['Primary Quote RIC'].tolist(),
        fields = ['TR.CommonName','TR.CompanyMarketCap', "TR.TRBCIndustryGroup"])

ind_const

,Instrument,Company Common Name,Company Market Cap,TRBC Industry Group Name
0,STAN.L,Standard Chartered PLC,19423385877.672001,Banking Services
1,CRDA.L,Croda International PLC,6745788203.02,Chemicals
2,ANTO.L,Antofagasta PLC,21669130156.099998,Metals & Mining
3,EZJ.L,Easyjet PLC,3867367147.55,Passenger Transportation Services
4,BNZL.L,Bunzl plc,10641359460.280001,Diversified Industrial Goods Wholesale
...,...,...,...,...
95,ULVR.L,Unilever PLC,105411638343.960007,Personal & Household Products & Services
96,OCDO.L,Ocado Group PLC,2988270943.2,Diversified Retail
97,LSEG.L,London Stock Exchange Group PLC,49981801009.400002,Investment Banking & Investment Services
98,TSCO.L,Tesco PLC,21419463198.82,Food & Drug Retailing


In [57]:
# Function to find top companies by market cap in each sector
def top_companies(group):
    return group.sort_values(by='Company Market Cap', ascending=False).head(1)

# Applying the function
top_per_sector = ind_const.groupby('TRBC Industry Group Name').apply(top_companies)

top_per_sector.reset_index(drop=True, inplace=True)
top_per_sector

,Instrument,Company Common Name,Company Market Cap,TRBC Industry Group Name
0,BAES.L,BAE Systems PLC,41682353681.0,Aerospace & Defense
1,HSBA.L,HSBC Holdings PLC,134517615980.369995,Banking Services
2,DGE.L,Diageo PLC,61262136403.379997,Beverages
3,CRDA.L,Croda International PLC,6745788203.02,Chemicals
4,SMT.L,Scottish Mortgage Investment Trust PLC,12317444408.048,Collective Investments
5,SMIN.L,Smiths Group PLC,5739725663.69,Consumer Goods Conglomerates
6,SKG.I,Smurfit Kappa Group PLC,11435953816.799999,Containers & Packaging
7,BNZL.L,Bunzl plc,10641359460.280001,Diversified Industrial Goods Wholesale
8,BMEB.L,B&M European Value Retail SA,5310780585.216,Diversified Retail
9,SSE.L,SSE PLC,19468936896.75,Electric Utilities & IPPs


In [58]:
ric = top_per_sector["Instrument"].to_list()

In [59]:
# for r in ric:
#     data, err = ek.get_data(r ,fields = [ek.TR_Field('tr.close'), ek.TR_Field('tr.close.date')], 
#                             parameters = {"SDate": "2000-01-01", "EDate": "2024-05-01"})
#     if not isinstance(data, pd.DataFrame):
#         data = pd.DataFrame(data)
#     portfolio = portfolio.append(data)
# portfolio = pd.DataFrame()  # Create an empty dataframe to store the portfolio data

portfolio = pd.DataFrame()  # Ensure portfolio is initialized as an empty DataFrame

for r in ric:
    data, err = ek.get_data(r, fields=[ek.TR_Field('tr.close'), ek.TR_Field('tr.close.date')], 
                            parameters={"SDate": "2000-01-01", "EDate": "2024-05-01"})
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)
    portfolio = pd.concat([portfolio, data], ignore_index=True)

portfolio.head()  # Display the updated portfolio dataframe


,Instrument,Price Close,Date
0,BAES.L,408.5,2000-01-04T00:00:00Z
1,BAES.L,401.25,2000-01-05T00:00:00Z
2,BAES.L,404.75,2000-01-06T00:00:00Z
3,BAES.L,388.25,2000-01-07T00:00:00Z
4,BAES.L,380.25,2000-01-10T00:00:00Z


In [60]:
portfolio.groupby('Instrument').count()

,Price Close,Date
Instrument,,
AZN.L,6147,6147
BAES.L,6147,6147
BATS.L,6147,6147
BKGH.L,6147,6147
BMEB.L,2499,6147
BNZL.L,6147,6147
CPG.L,5872,6147
CRDA.L,6147,6147
DGE.L,6147,6147


In [61]:
portfolio

,Instrument,Price Close,Date
0,BAES.L,408.5,2000-01-04T00:00:00Z
1,BAES.L,401.25,2000-01-05T00:00:00Z
2,BAES.L,404.75,2000-01-06T00:00:00Z
3,BAES.L,388.25,2000-01-07T00:00:00Z
4,BAES.L,380.25,2000-01-10T00:00:00Z
...,...,...,...
196729,SVT.L,2442.0,2024-04-25T00:00:00Z
196730,SVT.L,2458.0,2024-04-26T00:00:00Z
196731,SVT.L,2480.0,2024-04-29T00:00:00Z
196732,SVT.L,2467.0,2024-04-30T00:00:00Z


In [69]:
import pandas as pd

# Assuming the 'portfolio' DataFrame has been loaded with columns 'Date', 'Instrument', and 'Price Close'

# Convert 'Date' to datetime and normalize timezone if needed (remove timezone)
portfolio['Date'] = pd.to_datetime(portfolio['Date']).dt.tz_localize(None)

# Creating a full DataFrame assuming all combinations should exist
dates = pd.date_range(start='2000-01-01', end='2024-05-01', freq='D')
instruments = portfolio['Instrument'].unique()
full_index = pd.MultiIndex.from_product([dates, instruments], names=['Date', 'Instrument'])
full_df = pd.DataFrame(index=full_index).reset_index()

# Convert 'Date' in full_df to datetime and normalize timezone (since it's already without timezone, this is for demonstration)
full_df['Date'] = pd.to_datetime(full_df['Date'])

# Merge to find and handle missing data
merged_df = full_df.merge(portfolio, on=['Date', 'Instrument'], how='left')

# Check for missing data
missing_data = merged_df[merged_df['Price Close'].isna()]

# Fill missing values if necessary, here using forward fill as an example
# You could use `bfill` or `interpolate()` as well
merged_df['Price Close'].fillna(method='ffill', inplace=True)

# Now pivot the DataFrame to the desired format
pivot_df = merged_df.pivot(index='Date', columns='Instrument', values='Price Close')

# Displaying the first few rows of the pivoted DataFrame
pivot_df


Instrument,AZN.L,BAES.L,BATS.L,BKGH.L,BMEB.L,BNZL.L,CPG.L,CRDA.L,DGE.L,EXPN.L,...,SMIN.L,SMT.L,SN.L,SPX.L,SSE.L,SVT.L,TSCO.L,ULVR.L,VOD.L,WPP.L
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-01-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-01-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2000-01-04,2392.339763,408.5,332.25,522.974161,350.1837,350.1837,522.974161,282.758571,500.0,196.564953,...,911.563348,94.2,218.22864,444.355852,486.376112,625.338004,181.484681,1013.333232,295.186056,970.0
2000-01-05,2346.502026,401.25,335.5,516.637339,352.699388,352.699388,516.637339,295.77047,505.0,204.826379,...,901.451719,92.5,219.52762,437.334428,492.259694,634.263903,184.480301,1019.999898,280.914974,875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-27,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,...,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0
2024-04-28,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,...,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0
2024-04-29,12024.0,1355.0,2345.0,4750.0,517.0,3080.0,2216.0,4625.0,2769.5,3257.0,...,1617.0,833.2,984.0,8945.0,1676.5,2480.0,292.0,4105.0,70.0,817.2


In [70]:
portfolio_df = pivot_df.dropna()

In [71]:
portfolio_df

Instrument,AZN.L,BAES.L,BATS.L,BKGH.L,BMEB.L,BNZL.L,CPG.L,CRDA.L,DGE.L,EXPN.L,...,SMIN.L,SMT.L,SN.L,SPX.L,SSE.L,SVT.L,TSCO.L,ULVR.L,VOD.L,WPP.L
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,2392.339763,408.5,332.25,522.974161,350.1837,350.1837,522.974161,282.758571,500.0,196.564953,...,911.563348,94.2,218.22864,444.355852,486.376112,625.338004,181.484681,1013.333232,295.186056,970.0
2000-01-05,2346.502026,401.25,335.5,516.637339,352.699388,352.699388,516.637339,295.77047,505.0,204.826379,...,901.451719,92.5,219.52762,437.334428,492.259694,634.263903,184.480301,1019.999898,280.914974,875.0
2000-01-06,2341.625671,404.75,328.25,509.555009,368.29665,368.29665,509.555009,294.269097,550.5,222.346299,...,926.730791,90.4,217.189456,437.334428,505.252604,682.56877,184.480301,1053.333228,272.402399,867.5
2000-01-07,2397.216118,388.25,331.25,513.655306,363.768413,363.768413,513.655306,294.769554,565.0,235.877944,...,927.741954,91.3,218.22864,461.407883,510.891037,706.721203,181.983951,1104.444334,287.67496,900.0
2000-01-08,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,...,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203,706.721203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-27,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,...,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0
2024-04-28,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,...,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0,2458.0
2024-04-29,12024.0,1355.0,2345.0,4750.0,517.0,3080.0,2216.0,4625.0,2769.5,3257.0,...,1617.0,833.2,984.0,8945.0,1676.5,2480.0,292.0,4105.0,70.0,817.2


In [72]:
number_of_symbols = len(portfolio_df.columns)

# Calculate the Log of returns.
log_return = np.log(1 + portfolio_df.pct_change())

# Generate Random Weights.
random_weights = np.array(np.random.random(number_of_symbols))

# Generate the Rebalance Weights, these should equal 1.
rebalance_weights = random_weights / np.sum(random_weights)

# Calculate the Expected Returns, annualize it by multiplying it by `252`.
exp_ret = np.sum((log_return.mean() * rebalance_weights) * 252)

# Calculate the Expected Volatility, annualize it by multiplying it by `252`.
exp_vol = np.sqrt(
np.dot(
    rebalance_weights.T,
    np.dot(
        log_return.cov() * 252,
        rebalance_weights
    )
)
)

# Calculate the Sharpe Ratio.
sharpe_ratio = (exp_ret - 0.01)/ exp_vol

# Put the weights into a data frame to see them better.
weights_df = pd.DataFrame(data={
'random_weights': random_weights,
'rebalance_weights': rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print('-'*80)
print(weights_df)
print('-'*80)

# Do the same with the other metrics.
metrics_df = pd.DataFrame(data={
    'Expected Portfolio Returns': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])

print('')
print('='*100)
print('PORTFOLIO METRICS:')
print('-'*100)
print(metrics_df)
print('-'*100)


PORTFOLIO WEIGHTS:
--------------------------------------------------------------------------------
    random_weights  rebalance_weights
0         0.687457           0.043255
1         0.928706           0.058434
2         0.402382           0.025318
3         0.706147           0.044431
4         0.554912           0.034915
5         0.094932           0.005973
6         0.847454           0.053322
7         0.211720           0.013321
8         0.959429           0.060367
9         0.980285           0.061679
10        0.584915           0.036803
11        0.278744           0.017538
12        0.355018           0.022338
13        0.723410           0.045517
14        0.071282           0.004485
15        0.357933           0.022521
16        0.414183           0.026060
17        0.851809           0.053596
18        0.590685           0.037166
19        0.403485           0.025387
20        0.234337           0.014744
21        0.583618           0.036721
22        0.029772       

In [73]:
# Initialize the components, to run a Monte Carlo Simulation.

# We will run 5000 iterations.
num_of_portfolios = 5000

# Prep an array to store the weights as they are generated, 5000 iterations for each of our 4 symbols.
all_weights = np.zeros((num_of_portfolios, number_of_symbols))

# Prep an array to store the returns as they are generated, 5000 possible return values.
ret_arr = np.zeros(num_of_portfolios)

# Prep an array to store the volatilities as they are generated, 5000 possible volatility values.
vol_arr = np.zeros(num_of_portfolios)

# Prep an array to store the sharpe ratios as they are generated, 5000 possible Sharpe Ratios.
sharpe_arr = np.zeros(num_of_portfolios)

# Start the simulations.
for ind in range(num_of_portfolios):

    # First, calculate the weights.
    weights = np.array(np.random.random(number_of_symbols))
    weights = weights / np.sum(weights)

    # Add the weights, to the `weights_arrays`.
    all_weights[ind, :] = weights

    # Calculate the expected log returns, and add them to the `returns_array`.
    ret_arr[ind] = np.sum((log_return.mean() * weights) * 252)

    # Calculate the volatility, and add them to the `volatility_array`.
    vol_arr[ind] = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights))
    )

    # Calculate the Sharpe Ratio and Add it to the `sharpe_ratio_array`.
    sharpe_arr[ind] = (ret_arr[ind] - 0.01)/vol_arr[ind]

# Let's create our "Master Data Frame", with the weights, the returns, the volatility, and the Sharpe Ratio
simulations_data = [ret_arr, vol_arr, sharpe_arr, all_weights]

# Create a DataFrame from it, then Transpose it so it looks like our original one.
simulations_df = pd.DataFrame(data=simulations_data).T

# Give the columns the Proper Names.
simulations_df.columns = [
    'Returns',
    'Volatility',
    'Sharpe Ratio',
    'Portfolio Weights'
]

# Make sure the data types are correct, we don't want our floats to be strings.
simulations_df = simulations_df.infer_objects()

# Print out the results.
print('')
print('='*80)
print('SIMULATIONS RESULT:')
print('-'*80)
print(simulations_df.head())
print('-'*80)


SIMULATIONS RESULT:
--------------------------------------------------------------------------------
    Returns  Volatility  Sharpe Ratio  \
0  0.036060    7.398009      0.003523   
1  0.029975    7.407506      0.002697   
2  0.035417    6.637380      0.003829   
3  0.035055    7.267866      0.003447   
4  0.037490    7.673344      0.003583   

                                   Portfolio Weights  
0  [0.01865982313345805, 0.031870953400241424, 0....  
1  [0.04049626985550462, 0.025857062611285907, 0....  
2  [0.022391436706531813, 0.055026593495031806, 0...  
3  [0.026136940680494818, 0.04045331506276446, 0....  
4  [0.04226284024001345, 0.0119488066036346, 0.04...  
--------------------------------------------------------------------------------


In [74]:
# Return the Max Sharpe Ratio from the run.
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Return the Min Volatility from the run.
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('')
print('='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(max_sharpe_ratio)
print('-'*80)

print('')
print('='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(min_volatility)
print('-'*80)


MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns                                                       0.049137
Volatility                                                    6.653683
Sharpe Ratio                                                  0.005882
Portfolio Weights    [0.023857238688219807, 0.017027020332987898, 0...
Name: 1271, dtype: object
--------------------------------------------------------------------------------

MIN VOLATILITY:
--------------------------------------------------------------------------------
Returns                                                       0.043864
Volatility                                                    6.007147
Sharpe Ratio                                                  0.005637
Portfolio Weights    [0.03790404371836659, 0.054883757364254084, 0....
Name: 19, dtype: object
--------------------------------------------------------------------------------


In [75]:
def get_metrics(weights: list) -> np.array:
    """
    ### Overview:
    ----
    With a given set of weights, return the portfolio returns,
    the portfolio volatility, and the portfolio sharpe ratio.

    ### Arguments:
    ----
    weights (list): An array of portfolio weights.

    ### Returns:
    ----
    (np.array): An array containg return value, a volatility value,
        and a sharpe ratio.
    """

    # Convert to a Numpy Array.
    weights = np.array(weights)

    # Calculate the returns, remember to annualize them (252).
    ret = np.sum(log_return.mean() * weights) * 252

    # Calculate the volatility, remember to annualize them (252).
    vol = np.sqrt(
        np.dot(weights.T, np.dot(log_return.cov() * 252, weights))
    )

    # Calculate the Sharpe Ratio.
    sr = (ret - 0.01) / vol

    return np.array([ret, vol, sr])

def grab_negative_sharpe(weights: list) -> np.array:
    """The function used to minimize the Sharpe Ratio.

    ### Arguments:
    ----
    weights (list): The weights, we are testing to see
        if it's the minimum.

    ### Returns:
    ----
    (np.array): An numpy array of the portfolio metrics.
    """
    return get_metrics(weights)[2] - 1

def grab_volatility(weights: list) -> np.array:
    """The function used to minimize the Sharpe Ratio.

    ### Arguments:
    ----
    weights (list): The weights, we are testing to see
        if it's the minimum.

    ### Returns:
    ----
    (np.array): An numpy array of the portfolio metrics.
    """
    return get_metrics(weights)[1]

def check_sum(weights: list) -> float:
    """Ensure the allocations of the "weights", sums to 1 (100%)

    ### Arguments:
    ----
    weights (list): The weights we want to check to see
        if they sum to 1.

    ### Returns:
    ----
    float: The different between 1 and the sum of the weights.
    """
    return np.sum(weights) - 1

In [76]:
# Define the boundaries for each symbol. Remember I can only invest up to 100% of my capital into a single asset.
bounds = tuple((0, 1) for symbol in range(number_of_symbols))

# Define the constraints, here I'm saying that the sum of each weight must not exceed 100%.
constraints = ({'type': 'eq', 'fun': check_sum})

# We need to create an initial guess to start with,
# and usually the best initial guess is just an
# even distribution. In this case 25% for each of the 4 stocks.
init_guess = number_of_symbols * [1 / number_of_symbols]

# Perform the operation to minimize the risk.
optimized_sharpe = sci_opt.minimize(
    grab_negative_sharpe, # minimize this.
    init_guess, # Start with these values.
    method='SLSQP',
    bounds=bounds, # don't exceed these bounds.
    constraints=constraints # make sure you don't exceed the 100% constraint.
)

# Print the results.
print('')
print('='*80)
print('OPTIMIZED SHARPE RATIO:')
print('-'*80)
print(optimized_sharpe)
print('-'*80)


OPTIMIZED SHARPE RATIO:
--------------------------------------------------------------------------------
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.0047587636044806
       x: [ 8.265e-18  0.000e+00 ...  1.041e-16  0.000e+00]
     nit: 28
     jac: [ 2.452e-03  3.940e-03 ...  1.708e-03  1.500e-03]
    nfev: 924
    njev: 28
--------------------------------------------------------------------------------


In [77]:
# Grab the metrics.
optimized_metrics = get_metrics(weights=optimized_sharpe.x)

# Print the Optimized Weights.
print('')
print('='*80)
print('OPTIMIZED WEIGHTS:')
print('-'*80)
print(optimized_sharpe.x)
print('-'*80)


# Print the Optimized Metrics.
print('')
print('='*80)
print('OPTIMIZED METRICS:')
print('-'*80)
print(optimized_metrics)
print('-'*80)


OPTIMIZED WEIGHTS:
--------------------------------------------------------------------------------
[8.26482234e-18 0.00000000e+00 1.12290962e-18 0.00000000e+00
 0.00000000e+00 3.75383905e-18 0.00000000e+00 5.56200810e-19
 6.32173117e-18 0.00000000e+00 0.00000000e+00 3.56956209e-17
 1.00000000e+00 1.37156869e-18 3.96630794e-18 2.25251363e-19
 2.74063739e-18 6.51185901e-18 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 4.47744219e-18
 0.00000000e+00 5.07260165e-18 7.18164722e-18 6.41941141e-18
 1.66566788e-17 1.46798568e-17 1.04083409e-16 0.00000000e+00]
--------------------------------------------------------------------------------

OPTIMIZED METRICS:
--------------------------------------------------------------------------------
[-6.65404853e-02  1.60841117e+01 -4.75876360e-03]
--------------------------------------------------------------------------------


In [78]:
# Define the boundaries for each symbol. Remember I can only invest up to 100% of my capital into a single asset.
bounds = tuple((0, 1) for symbol in range(number_of_symbols))

# Define the constraints, here I'm saying that the sum of each weight must not exceed 100%.
constraints = ({'type': 'eq', 'fun': check_sum})

# We need to create an initial guess to start with,
# and usually the best initial guess is just an
# even distribution. In this case 25% for each of the 4 stocks.
init_guess = number_of_symbols * [1 / number_of_symbols]

# Perform the operation to minimize the risk.
optimized_volatility = sci_opt.minimize(
    grab_volatility, # minimize this.
    init_guess, # Start with these values.
    method='SLSQP',
    bounds=bounds, # don't exceed these bounds.
    constraints=constraints # make sure you don't exceed the 100% constraint.
)

# Print the results.
print('')
print('='*80)
print('OPTIMIZED VOLATILITY RATIO:')
print('-'*80)
print(optimized_volatility)
print('-'*80)


OPTIMIZED VOLATILITY RATIO:
--------------------------------------------------------------------------------
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 2.327944584256274
       x: [ 5.942e-02  6.998e-15 ...  1.174e-15  0.000e+00]
     nit: 10
     jac: [ 2.330e+00  3.482e+00 ...  5.663e+00  3.748e+00]
    nfev: 332
    njev: 10
--------------------------------------------------------------------------------


In [79]:
# Grab the metrics.
optimized_metrics = get_metrics(weights=optimized_volatility.x)

# Print the Optimized Weights.
print('')
print('='*80)
print('OPTIMIZED WEIGHTS:')
print('-'*80)
print(optimized_volatility.x)
print('-'*80)


# Print the Optimized Metrics.
print('')
print('='*80)
print('OPTIMIZED METRICS:')
print('-'*80)
print(optimized_metrics)
print('-'*80)


OPTIMIZED WEIGHTS:
--------------------------------------------------------------------------------
[5.94184775e-02 6.99829883e-15 4.62898179e-02 0.00000000e+00
 4.08377309e-15 0.00000000e+00 0.00000000e+00 1.20104935e-14
 5.18960914e-01 0.00000000e+00 0.00000000e+00 6.01519424e-16
 7.14003362e-15 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 2.61840246e-15
 5.09328703e-16 0.00000000e+00 3.75330790e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.92361860e-15 3.31423272e-17
 4.38003653e-15 0.00000000e+00 1.17432821e-15 0.00000000e+00]
--------------------------------------------------------------------------------

OPTIMIZED METRICS:
--------------------------------------------------------------------------------
[0.03638088 2.32794458 0.01133226]
--------------------------------------------------------------------------------
